In [80]:
import pyspark
import os
import numpy as np
import matplotlib.pyplot as plt

os.environ["PYSPARK_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'
os.environ["PYSPARK_DRIVER_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

## * 1) DataFrame을 생성

In [81]:
data = spark.read.text(os.path.join('data','제74주년 경찰의 날 기념식 축사.txt'))

In [82]:
data.show()

+----------------------------------+
|                             value|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
|                                  |
| 국민의 안전을 위해 밤낮없이 애...|
|                                  |
|오늘 홍조근정훈장을 받으신 중앙...|
|                                  |
|           사랑하는 경찰관 여러분,|
|                                  |
|여러분의 헌신적 노력으로 우리의...|
|                                  |
| 치안의 개선은 국민의 체감으로 ...|
|                                  |
| 한국을 찾는 외국 관광객들도 우...|
|                                  |
|   올해는 ‘경찰의 날’에 맞춰 국...|
|                                  |
|         자랑스러운 경찰관 여러분,|
|                                  |
| 경찰헌장은 “나라와 겨레를 위하...|
|                                  |
+----------------------------------+
only showing top 20 rows



## * 2) 단어로 분리해서, 출력

In [83]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="value", outputCol="words")
tokDf = tokenizer.transform(data)

In [84]:
tokDf.show()

+----------------------------------+--------------------------------+
|                             value|                           words|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|                                  |                              []|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
|                                  |                              []|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
|                                  |                              []|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|                                  |     

## * 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [85]:
stopwords = tokDf.select("words")\
    .rdd\
    .flatMap(lambda x:x).flatMap(lambda x:x)\
    .filter(lambda x: len(x) == 1)\
    .collect()
stopwords

['돌', '참', '더', '될', '그', '등', '그', '그', '법', '큰', '법', '더', '수', '그']

## * 4) 불용어 제거하고, 출력

In [86]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

stop.setStopWords(stopwords)
for e in stop.getStopWords():
    print (e, end="/")

돌/참/더/될/그/등/그/그/법/큰/법/더/수/그/

In [88]:
stopDf=stop.transform(tokDf)
stopDf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                             value|                           words|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|     [사랑하는, 경찰관, 여러분,]|
|                                  |                              []|                              []

## * 5) TF-IDF를 계산하고, 출력

In [104]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)



idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)

idfDf = idfModel.transform(hashDf)

for e in idfDf.select("idf").take(100):
    print(e)

Row(idf=SparseVector(262144, {162: 3.0445, 80732: 3.0445, 118171: 4.2565, 127225: 3.0445, 142775: 2.6391, 160086: 1.9459, 172380: 3.0445}))
Row(idf=SparseVector(262144, {249180: 0.6931}))
Row(idf=SparseVector(262144, {5341: 3.0445, 6304: 3.0445, 30732: 3.0445, 39431: 2.3514, 43098: 3.0445, 49855: 2.6391, 51468: 3.0445, 63600: 3.0445, 75300: 3.0445, 77757: 2.6391, 89318: 2.6391, 96799: 3.0445, 110980: 3.0445, 123553: 2.6391, 160081: 3.0445, 160086: 1.9459, 167255: 1.9459, 178931: 3.0445, 208192: 3.0445, 217323: 5.2781, 257249: 3.0445, 261393: 3.0445}))
Row(idf=SparseVector(262144, {249180: 0.6931}))
Row(idf=SparseVector(262144, {29823: 3.0445, 32228: 3.0445, 36822: 3.0445, 39431: 2.3514, 41144: 3.0445, 49855: 2.6391, 61014: 3.0445, 61103: 3.0445, 64713: 3.0445, 72971: 3.0445, 82902: 2.6391, 84159: 2.6391, 89227: 3.0445, 118725: 6.089, 153204: 3.0445, 167255: 1.9459, 178229: 3.0445, 185989: 3.0445, 206065: 3.0445, 217323: 2.6391, 235673: 3.0445, 237148: 3.0445, 248593: 3.0445, 249689: 3.

## * 6) TF-IDF 컬럼을 features로 구성, 출력

In [101]:
idfDf.select("nostops", "idf").show(100)

+--------------------------------+--------------------+
|                         nostops|                 idf|
+--------------------------------+--------------------+
|   [존경하는, 국민, 여러분,, ...|(262144,[162,8073...|
|                              []|(262144,[249180],...|
|  [국민의, 안전을, 위해, 밤낮...|(262144,[5341,630...|
|                              []|(262144,[249180],...|
|[오늘, 홍조근정훈장을, 받으신...|(262144,[29823,32...|
|                              []|(262144,[249180],...|
|     [사랑하는, 경찰관, 여러분,]|(262144,[118171,1...|
|                              []|(262144,[249180],...|
| [여러분의, 헌신적, 노력으로,...|(262144,[11202,28...|
|                              []|(262144,[249180],...|
|  [치안의, 개선은, 국민의, 체...|(262144,[883,9986...|
|                              []|(262144,[249180],...|
|  [한국을, 찾는, 외국, 관광객...|(262144,[7183,239...|
|                              []|(262144,[249180],...|
|    [올해는, ‘경찰의, 날’에, ...|(262144,[554,4988...|
|                              []|(262144,[249180],...|
|   [자랑스러운, 경찰관,